<a href="https://colab.research.google.com/github/man-do/scripts_and_notebooks/blob/main/face_detection_and_and_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow
from ipywidgets import Video, Image

In [ ]:
# Set up the workspace
!sudo apt install ffmpeg

if not os.path.isfile('test_video.mp4'):
  #!wget --quiet -O test_video.mp4 https://media.istockphoto.com/videos/black-man-and-white-man-hugging-and-smiling-equality-friendship-video-id1253066043
  !wget --quiet -O test_video.mp4 https://media.istockphoto.com/videos/two-businessmen-talking-while-walking-in-office-video-id1176752673

if not os.path.isfile('haarcascade_frontalface_default.xml'):
  !wget --quiet https://raw.githubusercontent.com/opencv/opencv/4.x/data/haarcascades/haarcascade_frontalface_default.xml

if not os.path.isfile('test_img.png'):
  !wget --quiet -O test_img.png https://images.ctfassets.net/16nm6vz43ids/4Ybw3hTeNyg3Av2TbeyhnX/745b812e0f050cd0f13ff5ca432e12d2/how-to-connect-two-pairs-of-airpods.png
  #!wget --quiet -O test_img.jpg https://image.shutterstock.com/image-photo/image-cute-young-loving-couple-260nw-1130187179.jpg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
face_cascade = cv2.CascadeClassifier()
model_location = cv2.samples.findFile('haarcascade_frontalface_default.xml')
face_cascade.load(model_location)
OPENCV_OBJECT_TRACKERS = {
	"CSRT": cv2.TrackerCSRT_create,
	"KCF": cv2.TrackerKCF_create,
	"BOOSTING": cv2.TrackerBoosting_create,
	"MIL": cv2.TrackerMIL_create,
	"TLD": cv2.TrackerTLD_create,
	"MEDIANFLOW": cv2.TrackerMedianFlow_create,
	"MOSSE": cv2.TrackerMOSSE_create,
	"GOTURN": cv2.TrackerGOTURN_create
}

In [ ]:
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
fontScale = 1
color_rect = (127,0,255)
color_text = (255,255,255)
thickness = 1
def draw_faces(frame, faces):
  i = 1
  if faces is ():
    frame = cv2.putText(frame, 'No faces found', (130,320), font,  3,color_text, thickness, 2)
  for (x,y,w,h) in faces:
    cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), color_rect, 2)
    frame = cv2.putText(frame, f'face {i}', (int(x),int(y+h-3)), font,  1,color_text, thickness, 2)
    i += 1
  return frame

In [ ]:
def detect_track_faces(video_file, tracker_name='MOSSE'):
  cap = cv2.VideoCapture(video_file)
  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  ok, frame = cap.read()
  w, h, ch = frame.shape
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
  faces = face_cascade.detectMultiScale(gray_frame, 1.0485258, 6)

  trackers = cv2.MultiTracker_create()
  trackers.add(OPENCV_OBJECT_TRACKERS[tracker_name](), frame, tuple(faces[0]))
  trackers.add(OPENCV_OBJECT_TRACKERS[tracker_name](), frame, tuple(faces[1]))

  frame = draw_faces(frame, faces)
  output_imgs = [frame]
  output_imgs.append(frame)
  out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30.0, (640, 360))
  out.write(frame)

  for i in range(length-1):
    ok, frame = cap.read()
    if not ok:
      break
    ok, faces = trackers.update(frame)
    frame = draw_faces(frame, faces)
    output_imgs.append(frame)
    out.write(frame)
  
  cap.release()
  out.release()
  

In [ ]:
detect_track_faces('test_video.mp4')

In [ ]:
!ffmpeg -hide_banner -loglevel error -i output_video.avi output_video.mp4 -y
Video.from_file('output_video.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x13@Mmdat\x00\x00\…